# Diplomado en Big Data - Introducción a las Herramientas de la Nube

### **Profesor:** Germán Leandro Contreras Sagredo
### **Ayudantes**
### Jorge Becerra de la Torre
### Valentina Rojas Mercier

## Actividad de la Clase 3: Bases de datos relacionales, nube virtual privada

La siguiente actividad mezclará la consola de AWS y la ejecución de código dentro del colab. Usaremos la consola para crear y eliminar una instancia de RDS y, por medio de código, crearemos tablas de datos, las poblaremos y las consultaremos.

# Algunas configuraciones iniciales

In [ ]:
# Instalamos las librerias a usar
!pip install psycopg2 boto3

import boto3
import psycopg2
import csv

In [ ]:
# Celda para obtener la IP de tu Colab
!curl ipecho.net/plain

Es importante mencionar algunos conceptos que verás a lo largo de la actividad:
1. `connection`: Esta clase de `psycopg2` maneja la conexión con la base de datos, que en este caso es la instancia de RDS que creaste - en esta variable almacenaremos la sesión a la que apunta. Puedes encontrar más información [acá](https://www.psycopg.org/docs/connection.html).
2. `cursor`: Es la clase que permite ejecutar las operaciones en SQL a la sesión de base de datos almacenada en `connection`. Para las mutaciones como `INSERT`, `CREATE TABLE` o `DROP TABLE`, es necesario llamar al método `commit` que esta expone, ya que en caso de no hacerlo, la operación no se llevará a cabo sino que se devolverá a su estado inicial mediante un _rollback_. Puedes encontrar más información [acá](https://www.psycopg.org/docs/cursor.html).

## DESAROLLO

En esta actividad deberás crear las siguientes tablas con sus respectivas columnas:
- Tabla "curso" *(nombre [text])*
- Tabla "profesor" *(nombre [text], edad [integer])*
- Tabla "estudiante" *(nombre [text], edad [integer], curso_id [integer])*
- Tabla "hace_clase_en" *(curso_id [integer], profesor_id [integer])*

La información para poblar cada tabla se encuentra en la sección `Archivos`.

La librería `psycopg2` es bastante directa en términos de cómo funciona: recibe una sentencia en SQL a ejecutar, ya sea para mutar tablas (crear, editar, eliminar) o solo para consultarlas. Te daremos para cada paso un ejemplo de cómo realizar esa operación específica en SQL, la que deberás rellenar con lo que corresponda en cada sección.



### Descarga de archivos

Para almacenar la ruta de los archivos, ejecuta las siguientes celdas.

In [ ]:
!gdown --id 1wU36DComfSy3su0Ugz_GmhKyBI2ixaEF
!gdown --id 1zxFSUZn3Vi5KaITx3gQeWdlfpaRzVbaL
!gdown --id 1Om0L53hqLNaahm_ayhGGwgkgMtxnkS42
!gdown --id 1M2vZWHSj7r2e_iGgSePtYLaXA_1EvctH

In [ ]:
# Estas serán las variables que deberás utilizar para la sección de cargar
# archivos CSV en el método que te fue entregado.
CURSO_CSV_FILENAME = 'curso - Hoja 1.csv'
ESTUDIANTE_CSV_FILENAME = 'estudiante - Hoja 1.csv'
PROFESOR_CSV_FILENAME = 'profesor - Hoja 1.csv'
HACE_CLASES_EN_FILENAME = 'hace_clases_en - Hoja 1.csv'

### Paso 1: Crear instancia de RDS por medio de la consola de AWS

1. Ingresa al servicio de RDS en AWS

![](https://drive.google.com/uc?export=view&id=11evZNb7CJ_aE4ivxlqZHk-aI0i-DJLBr)

2. Ingresa a crear una base de datos
![](https://drive.google.com/uc?export=view&id=1oeI_4X3iU1SLN0yxUJgDIWhAh6fo1OpP)

3. A continuación verás todas las configuraciones que debes aplicarle a la instancia, paso a paso. Primero el método y motor

![](https://drive.google.com/uc?export=view&id=1_Dac47cEGxW4mOlOZA9kcsdRtB8v0cmD)



4. Luego elige la plantilla de capa gratuita y elige el nombre que le pondrás
![](https://drive.google.com/uc?export=view&id=1KIhQad2W1_W5YDPkRFLgK91KpoNecK2v)

5. La instancia debe estar en una micro para que puedas seguir utilizando el free tier de AWS.
![](https://drive.google.com/uc?export=view&id=1_dhG8cwg2xJij1Kv2MnUhPGoUgKab4Yc)

6. Elige la VPC por defecto.
![](https://drive.google.com/uc?export=view&id=1sbtHn-qolUm5JvxjYx5vple5-PmsTB1v)

7. Continua con la autenticación
![](https://drive.google.com/uc?export=view&id=1QWdRyPciB2g7yLDMv0yFjNMx4UN6Ma3A)

8. Crea la instancia
![](https://drive.google.com/uc?export=view&id=1ns5DWcM5TyEhtxb5i5ZITzfhLVjT4pJw)

9. Luego, podrás ver tu instancia creada en la lista de instancias RDS de tu cuenta.
![](https://drive.google.com/uc?export=view&id=1A7TsYbb74s59gR2gpGFraCa2vE7RuuGC)

In [ ]:
# Almacena tus credenciales para acceder a tu instancia de RDS
RDB_AWS_ENDPOINT = ''
PORT = ''
USER_NAME = 'postgres'
PASSWORD = ''
DATABASE_NAME = 'postgres'

In [ ]:
# Creamos una conexión a nuestra base de datos
connection = psycopg2.connect(
    database=DATABASE_NAME,
    host = RDB_AWS_ENDPOINT,
    port = PORT,
    user = USER_NAME,
    password = PASSWORD
    )
cursor=connection.cursor()

### Paso 2: Crear tablas de datos
Antes de ver las funciones a utilizar en esta actividad, es necesario conocer los tipos de atributo o _data types_ de PostgreSQL. Estos los puedes consultar [aquí](https://www.postgresql.org/docs/9.5/datatype.html).

`execute_query:` A partir del argumento del tipo string `sql`, ejecutamos una _query_ SQL en nuestra base de datos. Particularmente, utilizaremos esta función para crear tablas. El formato recomendado es el siguiente.

```
create_table_sql = '''CREATE TABLE nombre_tabla (
    id SERIAL PRIMARY KEY,
    nombre_atributo tipo_atributo,
);'''
```

De aquí es importante dejar el atributo de id como `SERIAL`, ya que esto permitirá que el identificador incremente automáticamente con cada inserción. Si lo dejan como `INT`, tendrán que agregarlo manualmente en la tupla a insertar en la tabla.

In [ ]:
# Función para ejecutar un comando SQL.
def execute_query(sql):
  cursor.execute(sql)
  connection.commit()

`rollback:` Esta función deshace la última acción que hayas ejecutado (útil para cuando cometas un error que bloquee la realización de nuevas operaciones, debes deshacer aquella transacción antes).

In [ ]:
# Función para deshacer un commit
def rollback():
  cursor.execute("ROLLBACK")
  connection.commit()

In [ ]:
rollback()

In [ ]:
# Crear cada una de las tablas


### Paso 3: Poblar tablas
Te facilitamos 2 funciones, una para poblar cada una de las tablas que creaste en el paso anterior. Una te permite subir información por medio de un archivo csv, mientras que la otra te permite insertar una fila en la tabla.

`load_csv:` Recibe como parámetros el string del nombre del archivo CSV `csv_filename` (que puedes obtener de las variables dadas al inicio de este documento) con el que poblarás la tabla deseada; y el string `sql` con la sentencia SQL de inserción. El formato de esta debe ser el siguiente:

```
insert_tuple_sql = 'INSERT INTO nombre_tabla (atributo_1, atributo_2, ...) VALUES (%s, %s, ...)'
```

In [ ]:
# Función para poblar una tabla usando un csv
def load_csv(csv_filename, sql):
  csv_data = csv.reader(open(csv_filename))
  next(csv_data) # para saltarnos los headers
  for row in csv_data:
      cursor.execute(sql, row)
  connection.commit()

`insert_tuple:` Esta función inserta una fila en una tabla. Recibe como argumentos la tupla con los valores a insertar `attributes_tuple` (el que debe respetar el orden y formato de los atributos de la tabla); y el string `sql` correspondiente a la sentencia en SQL de la operación de inserción. Esta sigue el mismo formato utilizado en la función anterior.

In [ ]:
# Función para insertar una fila en una tabla
def insert_tuple(attribute_tuple, sql):
  cursor.execute(sql, attribute_tuple)
  connection.commit()

### Paso 3.1: Poblar tablas por medio de un archivo csv


In [ ]:
# Utiliza los archivos csv correspondientes a cada tabla para poblarlas

### Paso 3.2: Insertar fila en una tabla

In [ ]:
# Para cada tabla, inserta al menos una nueva fila

### Paso 4: Consultar información
A continuación te entregaremos 3 funciones para obtener datos de las distintas tablas que se encuentran en tu instancia de RDS.

`select_from_table`: A partir del string `table_name`, correspondiente al nombre de una tabla, obtiene todas sus filas y atributos.

In [ ]:
# Función para consultar una tabla
def select_from_table(table_name):
  sql = f'SELECT * FROM {table_name};'
  cursor.execute(sql)
  rows = cursor.fetchall()
  for row in rows:
    print(row)

`query_result`: A partir de un string `sql` correspondiente a una sentencia SQL, imprime todas las filas resultantes de la consulta. En este caso, el objetivo es obtener los resultados de un **cruce** entre tablas mediante un `INNER JOIN`. A continuación, un ejemplo:

```
inner_join_sql = 'SELECT tabla1.atributo_1, tabla1.atributo_2 FROM tabla1 INNER JOIN tabla2 ON tabla1.atributo = tabla2.atributo'
```

In [ ]:
# Función para cruzar información entre tablas
def query_result(sql):
  cursor.execute(sql)
  rows = cursor.fetchall()
  for row in rows:
    print(row)

`create_view`: Esta función tiene por objetivo crear una "View". Para ello, recibe como argumentos el string `view_name` correspondiente al nombre de la vista y el string `view_query_sql`, correspondiente a la consulta que generará la vista.

In [ ]:
# Función para realizar una consulta y guardarla en una "View"
def create_view(view_name, view_query_sql):
  sql = f'CREATE VIEW {view_name} AS {view_query_sql}'
  cursor.execute(sql)
  connection.commit()

### Paso 4.1: Consulta la información de cada una de las tablas

### Paso 4.2: Realiza el cruce entre la tabla curso y estudiante

### Paso 4.3: Realiza la consulta de la slide 12 de la clase y almacénala en una View

### Paso 4.4: Realiza una consulta a la view que creaste.

Estas se consultas como si fueran una tabla cualquiera. Puedes consultar las slides de la clase y utilizar los SQL de consulta que te facilitamos en la sección anterior.

In [ ]:
# Consulta la view que creaste

### Paso 5: Borrar tablas
A continuación se te solicitará borrar las tablas creadas. Lo importante es el orden en que las borras, pues existe cierta dependencia entre ellas y si no la respetas, no será posible eliminarlas. Para ello, utiliza la función facilitada a continuación.

`drop_table:` Elimina la tabla cuyo nombre le entregues a partir del argumento de tipo string `table_name`.

In [ ]:
def drop_table(table_name):
  cursor.execute(f"DROP TABLE {table_name} CASCADE;")
  connection.commit()

In [ ]:
# Elimina las tablas creadas


### Paso 6: Borrar instancia de RDS
Finalmente, para evitar cobros no deseados, te mostramos cómo borrar tu instancia de RDS por medio de la consola.

1. Presiona el botón "Acciones" habiendo seleccionado la instancia a borrar y escoge la opción "Eliminar".
![](https://drive.google.com/uc?export=view&id=12s05apOJBWq7KunHp48wW5Wkavkf9L1n)

2. Marca tu configuración deseada de _snapshots_ y _backups_ antes de eliminar de forma definitiva tu instancia.
![](https://drive.google.com/uc?export=view&id=1B0z_XkMuH-**C2yzLQknSeGIser3TaDovW**)

# Anexos

[PyMySQL](https://pypi.org/project/PyMySQL/), librería dedicada a bases de datos con el motor MySQL, útil si es que fuera el motor utilizado en nuestra instancia.